In [19]:
// boring imports & Stuff
var max = require('lodash/max');
var findIndex = require('lodash/findIndex');
var convnetjs, {Vol, Net, Layer, Trainer} = require('convnetjs')
var Plot = require('plotly-notebook-js');
var {grid2} = require('./utils')

function testOne(net, x, y) {
    var scores = net.forward(x);
    var maxp = max(scores.w);
    var result = { class: findIndex(scores.w, s => s === maxp), maxp, scores };
    result.correct = (y === result.class);
    return result;
}

function test(net, X, Y) {
    X.map((x, idx) => {
        var result = testOne(net, x);
        var rw = result.correct ? "RIGHT" : "WRONG"
        console.log("Pred:", result.class, "Actual:", Y[idx], "Prob:", result.maxp)
    });
}

'use strict'

# Neural Networks with Convnet.js continued...

We're continuing the XOR exercide from the last page with some concrete things to try. The first few cells just get us set up again.



#### Training Data

In [20]:
var X = [
    new Vol([0,0]),
    new Vol([1,0]),
    new Vol([0,1]),
    new Vol([1,1])
];

var Y = [0, 1, 1, 0];

#### Architecture

In [27]:
var layer_defs = [];

layer_defs.push({type:'input', out_sx:1, out_sy:1, out_depth:2});
layer_defs.push({type:'fc', num_neurons:4 , activation: 'relu'});
layer_defs.push({type:'softmax', num_classes:2});

// create a net
var net = new Net();
net.makeLayers(layer_defs);

#### Training

In [28]:
var trainer = new Trainer(net, { learning_rate:0.01, l2_decay:0.001 });

var NUM_PASSES = 1000;

for (let p = 0; p < NUM_PASSES; p++) {
    for (let i = 0; i < X.length; i++) {
        trainer.train(X[i], Y[i]); 
    }
}

console.log("Trained", NUM_PASSES*X.length, "iterations")


Trained 4000 iterations


#### Testing

In [29]:
test(net, X, Y)

Pred: 0 Actual: 0 Prob: 0.9936578022738751
Pred: 1 Actual: 1 Prob: 0.9957826764855495
Pred: 1 Actual: 1 Prob: 0.9959455822485174
Pred: 0 Actual: 0 Prob: 0.9964252884927225


Our network as is is the minimum to run convnet but it has practicaly no discriminatory power. We need to add another fully connected layer to the architecture.

check the [docs](http://cs.stanford.edu/people/karpathy/convnetjs/docs.html) and add a fully connected layer with 10 neurons and `relu` activation functions.


### Plot the decision space

Just like we did for k-means let's push some inputs through the network over the range of [-1, 1] in both input dimensions and plot the resulting class outputs as a heatmap.

Also it might be interetsting to plot out the probabilities or separate charts.

HINT: the testOne function in the top cell might help.

In [23]:
var G = grid2([-1,1], 0.1);

// compute the "heat" data
var H = G.map(col => {
    return col.map(x => {
        return 0;
    })
});

var data = {
    z: H,
    type: "heatmap"
}

var layout = {
    title: "decision space",
    width: 700,
    height: 700
}

$$html$$ = Plot.createPlot([data], layout).render()

#### Playtime

Things to try:
 - different numbers of neurons
 - different activation functions
 - more than 1 fc layer